<h3> Description of the Notebook </h3> 
This is some text descriping the work done by this notebook.

In [36]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [23]:
# import packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import re
import time

<h3> Specifying the parameters </h3>
In the next cell, we present all parameters that might have an effect on the search results, including:<br>
1. searching keyword lexicon<br>
2. academic databases<br>
3. initial urls when searching academic databases<br>
4. seed paper list for spanning citations<br>
5. conenctome database<br>
6. seaching queries of the connectome database<br>
7. on-topic keyword lexicon<br>
8. weights of on-topic keywords when calculating relatedness of a literature<br>
9. ChatGPT queries for relatedness of topic<br>
10. meta categories when extracting information of related literature<br>
11. keywords for searching meta categories<br>
12. ChatGPT queries for extracting information of meta categories of related literature<br>

In [7]:
# parameters

# searching keywords lexicon
search_kws_lexicon = "macaque AND (thalamus OR thalamocortical OR thalamo-cortical)" # in all fields

# academic databases
# Google Scholar: "https://scholar.google.com/"
# macaque thalamus OR thalamocortical OR thalamo-cortical
# 78100 results
# Web of Science: "https://www.webofscience.com/wos/woscc/advanced-search" # can be exported to excel file
# (ALL=(thalamus) OR ALL=(thalamocortical) OR ALL=(thalamo-cortical)) AND ALL=(macaque)
# 880 results
# PubMed Central PMC: "https://pubmed.ncbi.nlm.nih.gov/advanced/" # can be exported to .csv file
# ((thalamus) OR (thalamocortical) OR (thalamo-cortical)) AND (macaque)
# 2448 results
# Europe PMC = "https://europepmc.org/advancesearch" # search resuts can be exported to .csv file
# ("macaque") AND ("thalamus" OR "thalamocortical" OR "thalamo-cortical") AND (LANG:"eng" OR LANG:"en" OR LANG:"us")
# 5130 results
acad_dbs = ["Google Scholar", "Web of Science", "PubMed Central PMC", "Europe PMC"]

# initial urls for specified searching keyword lexicon and all academic databases
init_urls = {
    "gs": "https://scholar.google.com/scholar?start=0&q=macaque+thalamus+OR+thalamocortical+OR+thalamo-cortical&hl=en&as_sdt=1,5",
    "wos": "https://www.webofscience.com/wos/woscc/summary/3a00a41f-3135-4142-a950-c8d6eb3b20a7-99be93b8/relevance/1",
    "pmc": "https://pubmed.ncbi.nlm.nih.gov/?term=((thalamus)%20OR%20(thalamocortical)%20OR%20(thalamo-cortical))%20AND%20(macaque)&sort=relevance&page=1",
    "eupmc": "https://europepmc.org/search?query=%28%22macaque%22%29%20AND%20%28%22thalamus%22%20OR%20%22thalamocortical%22%20OR%20%22thalamo-cortical%22%29%20AND%20%28LANG%3A%22eng%22%20OR%20LANG%3A%22en%22%20OR%20LANG%3A%22us%22%29&page=1"
}

# seed literature list
seed_litera_list = []

# cocomac literature list
cocomac_litera_list = []

In [38]:
def search_google_scholar(init_url):
    # create a .txt file to record the urls of google scholar search results, clear the file if already exists
    f = open(fpath.poten_litera_gs, "w")
    f.truncate()
    f.close()

    # request the first page and extract the number of pages of the search results
    first_page = init_url
    response = requests.get(first_page, headers = plib.headers)
    soup = BeautifulSoup(response.content, "lxml")
    # print(soup)
    num_results_str_list = soup.select("div", {"class": "gs_ab_mdw"})
    for item in num_results_str_list:
        if "results" in item.get_text():
            num_results_str = item.get_text().split()
    num_results_str = num_results_str[1]
    # print(num_results_str)
    # print(int(num_results_str))
    num_results = int(re.sub(r"[^\w\s]", "", num_results_str))
    pages = int(num_results/10)
    print("Google Scholar searched " + str(num_results) + " results" + " displayed in " + str(pages) + " pages.")
    
    # iterate all pages and record the results
    # pages = 5
    for page in range(pages):
        time.sleep(random.randint(1, 5))
        time.sleep(random.randint(5, 10))
        start = page * 10
        page_url = init_url.split("?start=")[0] + "?start=" + str(start) + "&q=" + init_url.split("?start=")[1].split("&q=")[1]
        # print(page_url)
        # search a page
        response = requests.get(page_url, headers = plib.headers)
        soup = BeautifulSoup(response.content, "lxml")
        # print(soup)
        # print(soup.select("[data-lid]")) 
        for item in soup.select("[data-lid]"):
            # print(item)
            try:
                add_title = item.select("h3")[0].select("a", href = True)[0].get_text()
                # print(add_title)
            except:
                add_title = "not found"
            try:
                add_url = item.select("h3")[0].select("a", href = True)[0]["href"]
                # print(add_url)
            except:
                add_url = "not found"
            try:
                add_full_text_link = item.find_all("div", {'class': "gs_or_ggsm"})[0].find_all("a", href = True)[0]["href"]
                # print("1")
                # print(add_full_text_link)
            except:
                add_full_text_link = "not found"
            # print(add_full_text_link)
            row = {
                "title": [add_title],
                "url": [add_url],
                "full_text_url": [add_full_text_link]
            }
            columns = ["title", "url", "full_text_url"]
            plib.add_row_to_csv(fpath.poten_litera_gs, row, columns)
    print("Searching Google Scholar complated!")

def search_webofscience(init_url, headers):
    print("Searching Web of Science complated!")

def search_PubMed_Central_PMC(iinit_url, headers):
    print("Searching PubMedd Central PMC complated!")

def search_Europe_PMC(init_url, headers):
    print("Searching Europe PMC complated!")

# --------------------start of test code--------------------
# init_url = init_urls["gs"]
# headers = plib.headers
# search_google_scholar(init_url)
# ---------------------end of test code---------------------

Google Scholar searched 78100 results displayed in 7810 pages.
Searching Google Scholar complated!


In [ ]:
def search_webofscience(init_url):
    # search in the website and export the search results
    print("Searching Web of Science complated!")

In [ ]:
def search_pmc(init_url):
    # search in the website and export the search results
    print("Searching PubMedd Central PMC complated!")

In [ ]:
def search_eupmc(init_url):
    # search in the website and export the search results
    print("Searching Europe PMC complated!")

In [ ]:
# search academic databases, record the urls as a line in a .txt file from the webpages
def search_acad_dbs(acad_dbs, init_urls):
    try:
        for acad_db in acad_dbs:
            if acad_db == "Google Scholar":
                print("Searching Google Scholar...")
                search_google_scholar(init_urls["gs"])
            elif acad_db == "Web of Science":
                print("Searching Web of Science...")
                search_webofscience(init_urls["wos"])
            elif acad_db == "PubMed Central PMC":
                print("Searching PubMed Central PMC...")
                search_pmc(init_urls["pmc"])
            elif acad_db == "Europe_PMC":
                print("Searching Europe PMC...")
                search_eupmc(init_urls["eupmc"])
            else:
                print("Searching the specified academic database: " + acad_db + " is not supported by this function.")
                print("Plese choose one of the following databases:",)
                for db in ["Google Scholar", "Web of Science", "PubMed Central PMC", "Europe PMC"]:
                    print(db)
        return True
    except:
        return False


def span_citations(seed_litera_list, num_span_time):
    None
    
def search_conne_db():
    None

# --------------------start of test code--------------------
# test code
# ---------------------end of test code---------------------

In [ ]:
def merge_search_results():
    try:
        # google scholar search results
        # not processed for now

        # process web of science search results


        # process pmc search results


        # process eupmc search results


        # process seed literature citation spanning results
        # not processed for now
        # process connctome database literature results
        # not processed for now
        with open(fpath.gs_poten_urls, "r") as file:
            lines = []
            for line in file:
                print(line)
                line = line.strip()
                lines.append(line)
        print(len(lines))
        doi_list = []
        for url in lines:
            response = requests.get(url, headers = plib.headers)
            soup = BeautifulSoup(response.content,"lxml")
            # print(soup)
            num_results_str = soup.select("a", href = True)
            for href in num_results_str:
                if "//doi.org/" in href:
                    doi_list.append(href)
        doi_df = pd.DataFrame({"DOI": doi_list})
        plib.clear_file(fpath.path_poten_csv)
        doi_df.to_csv(fpath.path_poten_csv, index=False)
        
        # process wos_poten_urls
        doi_df = pd.read_csv(fpath.wos_poten_urls, sep=";")
        # print(doi_df.columns)
        # print(doi_df.head())
        doi_df = doi_df[["DOI"]]
        doi_df.to_csv(fpath.path_poten_csv, mode="a", index=False, header=False)
        
        # process pubmed_pmc_poten_urls
        doi_df = pd.read_csv(fpath.pubmed_pmc_poten_urls)
        doi_df = doi_df[["DOI"]]
        doi_df.to_csv(fpath.path_poten_csv, mode="a", index=False, header=False)
        
        # process eupmc_poten_urls
        doi_df = pd.read_csv(fpath.eupmc_poten_urls)
        doi_df = doi_df[["DOI"]]
        doi_df.to_csv(fpath.path_poten_csv, mode="a", index=False, header=False)
        
        # eliminate duplicates
        doi_df = pd.read_csv(fpath.path_poten_csv)
        print(len(doi_df))
        doi_df = doi_df.drop_duplicates(subset = "DOI")
        print(len(doi_df))
        doi_df.to_csv(fpath.path_poten_csv, index=False)
        # end of merge_search_results
        return True
    except:
        return False


# --------------------start of test code--------------------
# test code
# ---------------------end of test code---------------------

<h3> Main program </h3> 

In [ ]:
# first we need to search all related literature that might include data or information of thalamocortical connections
# search for potentially related literature using the listed 3 methods

# method 1: search acdemic databases using keywords
if search_acad_dbs(acad_dbs, init_urls):
    print("Searching academic databases completed!")
else:
    print("Attention! Something went wrong when searching academic databases completed!")

In [ ]:
# # method 2: spanning citations of seed papers
# if span_citations(seed_papers, num_span_time):
#     print("Spanning citations of seed literature list completed!")
# else:
#     print("Attention! Something went wrong when spanning citations of seed literature list!")


In [ ]:
# # method 3: search existing connectome databases
# if search_conne_db(connec_db, connec_db_quries):
#     print("Searching connectome databases completed!")
# else:
#     print("Attention! Something went wrong when searching connectome databases!")

In [ ]:
# merge all results
if merge_search_results():
    print("Merging all results completed!")
else:
    print("Attention! Something went wrong when merging all results completed!")

Now we have a list of actually related literature stored in list_of_related_literature.txt

Next step: we perform a information search on the list of related literature
We have a list of actually related literature at the moment, now we need to extract information we need from the literature. We intend to achieve this with a combination of automated searching and manual extraction

In [ ]:
# some test code, should comment-out

In [ ]:
# # test the redirect of the urls
# headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9"}

# response_pdf = requests.get("https://doi.org/10.1016/j.neuron.2020.01.005", allow_redirects=True, headers = headers)
# print(response_pdf.history)
# print(response_pdf.url)
# response_pdf_1 = requests.get("https://linkinghub.elsevier.com/retrieve/pii/S0896627320300052", allow_redirects=True, headers = headers)
# print(response_pdf_1.history)
# print(response_pdf_1.url)


# response_pdf = requests.get("https://onlinelibrary.wiley.com/doi/10.1111/ejn.13910", headers = headers)
# soup_pdf = BeautifulSoup(response_pdf.content,"lxml")
# print(soup_pdf)

In [ ]:
# import re
# if "//doi.org/" in "https://doi.org/10.1016/0165-0173(96)00003-3":
#     print("yes")

In [ ]:
# test extract doi from url
with open(fpath.gs_poten_urls, "r") as file:
    lines = []
    for line in file:
        print(line)
        line = line.strip()
        lines.append(line)
print(len(lines))
doi_list = []
for url in lines:
    response = requests.get(url, headers = plib.headers)
    soup = BeautifulSoup(response.content,"lxml")
    # print(soup)
    num_results_str = soup.select("a", href = True)
    print(num_results_str)
    for href in num_results_str:
        if "//doi.org/" in href["href"]:
            doi_list.append(href["href"])
            print(href["href"])
        else:
            print("Ops! Did't find DOI on this page!")

In [ ]:
# find DOI
# this link does not have "DOI" in href form but text from
url = "https://www.jneurosci.org/content/28/43/11042.short"
# url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2613515/"
response = requests.get(url, headers = plib.headers)
soup = BeautifulSoup(response.content,"lxml")
# print(soup)
doi_list = []
num_results_str = soup.select("a", href = True)
# print(num_results_str)
for item in num_results_str:
    if "//doi.org/" in item["href"]:
        print(item["href"])
        doi_list.append(item["href"].split("//doi.org/")[1])

print(doi_list)
        
if len(doi_list) == 0:
    print("Ops! Did't find DOI on this page!")

In [ ]:
# test redirect when access the doi link
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elsclient import ElsClient
import json
headers = {
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9", 
    "X-ELS-APIKEY": "310946e6e005957982c2c9cad6833ad3",
    "Accept": "application/pdf",
    "X-ELS-Insttoken": "instToken",
    "view": "FULL"
} 
# url = "https://www.jneurosci.org/content/28/43/11042.short"
 #url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2613515/"

# Journal of Neurophysiology
# url = "https://doi.org/10.1152/jn.2001.85.1.219"
# url = "https://journals.physiology.org/doi/10.1152/jn.2001.85.1.219"

# science direct
# url = "https://doi.org/10.1016/j.biopsych.2004.10.014"
# url = "https://linkinghub.elsevier.com/retrieve/pii/S0006322304010947"
# url = "https://www.sciencedirect.com/science/article/pii/S0006322304010947?via%3Dihub"
url = "https://api.elsevier.com/content/article/doi/{10.1016/j.biopsych.2004.10.014}"

# response = requests.get(url, headers = headers)
# soup = BeautifulSoup(response.content,"lxml")
# print(soup)
# print(response.history)
# print(response.url)
# # Load configuration
# con_file = open("config.json")
# config = json.load(con_file)
# con_file.close()

# response = requests.get(url, headers = headers)
# print(response)

# ## Initialize client
# client = ElsClient(config["apikey"])

# ## ScienceDirect (full-text) document example using DOI
# doi_doc = FullDoc(doi = "10.1016/j.biopsych.2004.10.014")
# print(doi_doc)
# if doi_doc.read(client):
#     print ("doi_doc.title: ", doi_doc.title)
#     doi_doc.write("doi_doc")   
# else:
#     print ("Read document failed.")

In [ ]:
# test pdf to text
from PyPDF2 import PdfReader
  
# creating a pdf reader object
reader = PdfReader("/Users/didihou/Downloads/fncir-09-00079.pdf")
  
# printing number of pages in pdf file
print(len(reader.pages))
  
# getting a specific page from the pdf file
page = reader.pages[0]
  
# extracting text from page
text = "".join(page.extract_text().splitlines())
print(text)

In [ ]:
# test request
url = 'https://journals.physiology.org/doi/pdf/10.1152/jn.2001.85.1.219'

response = requests.get(url, headers = plib.headers)
soup = BeautifulSoup(response.content,"lxml")
print(soup)

In [ ]:
url = 'https://journals.physiology.org/doi/epdf/10.1152/jn.2001.85.1.219'
r = requests.get(url, headers = plib.headers) 
print(r.url) 

In [35]:
df = pd.read_csv(fpath.poten_litera_gs)
print(df["full_text_url"])

KeyError: 'full_text_url'